Prep Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.sets import load_sets

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_sets()

In [4]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [5]:
train_dataset.y_tensor

tensor([65., 44., 25.,  ..., 13., 12., 29.])

Baseline Model

In [6]:
from src.models.null import NullModel
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [7]:
from src.models.performance import print_class_perf
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.07444192974099043
F1 Training: 0.010315310209270104


Define Architecture

This model tests trial and error for different neural network architectures

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 512)
        self.layer_2 = nn.Linear(512, 256)
        self.layer_3 = nn.Linear(256, 128)
        self.layer_out = nn.Linear(128, 104)
        self.softmax = nn.Softmax(dim=1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(256)
        self.batchnorm3 = nn.BatchNorm1d(128)
    

    def forward(self, x):
#         x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
#         x = F.dropout(F.relu(self.layer_2(x)), training=self.training)
#         x = self.layer_out(x)
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        return x
    
model = PytorchMultiClass(X_train.shape[1])

In [10]:
from src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

Train Model

Define 2 dictionaries that will store the accuracy/epoch and loss/epoch for both train and validation sets.

In [11]:
accuracy_stats = {'train': [], "val": []}
loss_stats = {'train': [], "val": []}

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [13]:
N_EPOCHS = 30
BATCH_SIZE = 10000

In [17]:
#use train_classification, test_classification defined in lab5
from src.models.pytorch import train_classification, test_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device, scheduler=scheduler, accuracy_stats=accuracy_stats, loss_stats=loss_stats, shuffle=True)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device, accuracy_stats=accuracy_stats, loss_stats=loss_stats)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0003	|	Acc: 21.6%
	(valid)	|	Loss: 0.0003	|	Acc: 22.8%
Epoch: 1
	(train)	|	Loss: 0.0003	|	Acc: 21.9%
	(valid)	|	Loss: 0.0003	|	Acc: 23.2%
Epoch: 2
	(train)	|	Loss: 0.0003	|	Acc: 22.1%
	(valid)	|	Loss: 0.0003	|	Acc: 23.5%
Epoch: 3
	(train)	|	Loss: 0.0003	|	Acc: 22.3%
	(valid)	|	Loss: 0.0003	|	Acc: 23.7%
Epoch: 4
	(train)	|	Loss: 0.0003	|	Acc: 22.5%
	(valid)	|	Loss: 0.0003	|	Acc: 24.1%
Epoch: 5
	(train)	|	Loss: 0.0003	|	Acc: 22.7%
	(valid)	|	Loss: 0.0003	|	Acc: 24.4%
Epoch: 6
	(train)	|	Loss: 0.0003	|	Acc: 22.9%
	(valid)	|	Loss: 0.0003	|	Acc: 24.6%
Epoch: 7
	(train)	|	Loss: 0.0003	|	Acc: 23.1%
	(valid)	|	Loss: 0.0003	|	Acc: 24.9%
Epoch: 8
	(train)	|	Loss: 0.0003	|	Acc: 23.3%
	(valid)	|	Loss: 0.0003	|	Acc: 25.0%
Epoch: 9
	(train)	|	Loss: 0.0003	|	Acc: 23.3%
	(valid)	|	Loss: 0.0003	|	Acc: 25.3%
Epoch: 10
	(train)	|	Loss: 0.0003	|	Acc: 23.5%
	(valid)	|	Loss: 0.0003	|	Acc: 25.4%
Epoch: 11
	(train)	|	Loss: 0.0003	|	Acc: 23.6%
	(valid)	|	Loss: 0.0003	|	Acc: 25.6%
Ep

In [18]:
torch.save(model, "../models/pytorch_nn_v3.pt")

In [20]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device, accuracy_stats=accuracy_stats, loss_stats=loss_stats)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.0003	|	Accuracy: 0.3


Model1 Potential Reasons to low score:
1. Imbalanced data
2. Only 1 hidden layer constructed
3. Too many classes

Adding more layer of hidden layers are not working better, need further research.

Trials:
    standard: 29-30%
    minmax: 29-30
    Robust: